In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./python.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

155


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

433

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeder = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [5]:
vector_1 = embeder.embed_query(all_splits[0].page_content)
vector_2 = embeder.embed_query(all_splits[1].page_content,output_dimensionality=768) 

In [6]:
print((vector_1))

[-0.01927693746984005, 0.0011522650020197034, -0.005431035533547401, -0.05403294414281845, 0.012089746072888374, 0.008886780589818954, -0.005132238380610943, 0.012012854218482971, 0.00579322362318635, 0.0044417306780815125, -0.020653748884797096, -0.010194954462349415, 0.008080662228167057, -0.00814838893711567, 0.12736470997333527, 0.003381297690793872, -0.005308378022164106, -0.028391074389219284, -0.010937056504189968, -0.011530962772667408, 0.004363138694316149, 0.017584020271897316, -0.00716739846393466, -0.01030760258436203, 0.02645196206867695, 0.0022041616030037403, 0.022319165989756584, 0.0063917795196175575, 0.008499450981616974, -0.007446779869496822, -0.028820015490055084, 0.024114523082971573, 0.00393947446718812, 0.02867339737713337, -0.014215916395187378, 0.002299416810274124, 0.0028217073995620012, 0.0062240660190582275, -0.017491651698946953, -0.006120436359196901, -0.017171798273921013, 0.008678303100168705, -0.001428723451681435, -0.009712796658277512, -0.00157890818

In [9]:
print(len(vector_2))

768


In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeder)

In [18]:
ids = vector_store.add_documents(documents=all_splits[0:100])

In [16]:
print(ids)

['63c4e11e-3fdf-443e-8755-b36b2252080a', 'dc7546ec-d407-4191-99d6-076e1c12b9eb', '094c9c4c-7a38-423a-95b1-ba6ce7dc7c47', '3f98968b-065f-4228-8c3a-fb09a892177e', 'be43d418-dde5-4175-b325-f8f1ef0fa303', 'ff450a70-bfa0-4df5-ac9e-5267345ef70f', 'fac02a74-3570-41d7-ae2c-06fbcacd98ca', '50dc2c19-aa31-4eca-9730-8b4b5c9a51dd', '6a6cde0f-75c5-42b3-86f2-31058dffac45', '6f477d10-760e-4d16-b90d-11018cfada34']


In [ ]:
results = vector_store.similarity_search(
    "how to write Strings in python",
)

# doc, score = results[0]
print(results[0])

In [26]:
print((results))

[(Document(id='6bf965b3-8611-4466-b210-0f91b42808ac', metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-09-02T00:46:10+00:00', 'author': 'Guido van Rossum, and the Python development team', 'moddate': '2018-09-02T11:19:20-07:00', 'title': 'Python Tutorial', 'source': './python.pdf', 'total_pages': 155, 'page': 17, 'page_label': '12', 'start_index': 767}, page_content=">>> text\n'Put several strings within parentheses to have them joined together.'\nThis only works with two literals though, not with variables or expressions:\n>>> prefix = 'Py'\n>>> prefix 'thon' # can't concatenate a variable and a string literal\n...\nSyntaxError: invalid syntax\n>>> ('un' * 3) 'ium'\n...\nSyntaxError: invalid syntax\nIf you want to concatenate variables or a variable and a literal, use+:\n>>> prefix + 'thon'\n'Python'\nStrings can beindexed (subscripted), with the ﬁrst character having index 0. There is no separate character\ntype; a character i

In [ ]:
pip install -qU langchain-qdrant

In [10]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient("http://localhost:6333/")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeder,
)

In [ ]:
ids = vector_store.add_documents(documents=all_splits[0:100])

In [15]:
results = vector_store.similarity_search(
    "how to write Strings in python",
)

# doc, score = results[0]
print(results[0])

page_content='>>> text
'Put several strings within parentheses to have them joined together.'
This only works with two literals though, not with variables or expressions:
>>> prefix = 'Py'
>>> prefix 'thon' # can't concatenate a variable and a string literal
...
SyntaxError: invalid syntax
>>> ('un' * 3) 'ium'
...
SyntaxError: invalid syntax
If you want to concatenate variables or a variable and a literal, use+:
>>> prefix + 'thon'
'Python'
Strings can beindexed (subscripted), with the ﬁrst character having index 0. There is no separate character
type; a character is simply a string of size one:
>>> word = 'Python'
>>> word[0] # character in position 0
'P'
>>> word[5] # character in position 5
'n'
Indices may also be negative numbers, to start counting from the right:
12 Chapter 3. An Informal Introduction to Python' metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-09-02T00:46:10+00:00', 'author': 'Guido van Rossum, and the Pyth